## Recurrence, Depth and High-dimensional data
# Machine learning basics notebook

In this notebook we present basic methods to train and evaluate the performance of machine learning models. In order to keeps things simple, we'll use a simple linear model, called *polynomial regression*, to fit data that was generated synthetically.

The following elements will be presented:

* polynomial regression
* train and test sets
* train and validation loss
* cross-validation for model comparison
* regularization by dataset size

**References:**
* [Pattern recognition and machine learning](http://www.springer.com/gp/book/9780387310732), chapter 1, Bishop, Christopher M, springer, 2006.

*Please execute the cell bellow in order to initialize the notebook environment*

In [ ]:
%autosave 0
# %matplotlib inline
%matplotlib notebook

from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import mod3

plt.rcParams.update({'figure.figsize': (5.0, 4.0), 'lines.linewidth': 2.0})

## Polynomial regression

Suppose a process generated $n$ data points $\mathcal{D}=\{(x_1,y_1),\ldots,(x_n,y_n)\}$, where each $y_i$ is a function of $x_i$ and noise $\xi_i$. We write $y_i=f(x_i, \xi_i)$ for the unknown function $f()$ that models the response of the process, and the goal is to estimate $\hat{y}$ for any value of $x$.

Polynomial regression assumes the data generation process can be modeled as polynomial of $x$ with an additive noise term $\xi$. For a polynomial of degree $p$ this is written as:
$$
y = w_0 + w_1 x + w_2 x^2 + w_3 x^3 + \cdots + w_p x^p + \xi 
$$

For all data points in $\mathcal{D}$, this can be written as a system of linear equations:

$$
{\begin{bmatrix}y_{1}\\y_{2}\\y_{3}\\\vdots \\y_{n}\end{bmatrix}}={\begin{bmatrix}1&x_{1}&x_{1}^{2}&\dots &x_{1}^{p}\\1&x_{2}&x_{2}^{2}&\dots &x_{2}^{p}\\1&x_{3}&x_{3}^{2}&\dots &x_{3}^{p}\\\vdots &\vdots &\vdots &&\vdots \\1&x_{n}&x_{n}^{2}&\dots &x_{n}^{p}\end{bmatrix}}{\begin{bmatrix}w_{0}\\w_{1}\\w_{2}\\\vdots \\w_{p}\end{bmatrix}}+{\begin{bmatrix}\xi _{1}\\\xi _{2}\\\xi _{3}\\\vdots \\\xi _{n}\end{bmatrix}}
$$

This can be written in matrix notation as $\mathbf{y}=\mathbf{X}\mathbf{w}+\boldsymbol{\xi}$, where $\mathbf{y}$ and $\boldsymbol{\xi}$ are $n×1$ vectors, $\mathbf{X}$ is an $n×p$ matrix, and $\mathbf{w}$ is a $p×1$ vector of unknown parameters. The matrix $\mathbf{X}$ is sometimes called the *design matrix*.

The ordinary least squares (OLS) solution minimizes the sum of squared residuals 

$$C(\mathcal{D},\mathbf{w})=\frac{1}{2n}\sum _{i=1}^{n}(y_{i}-\mathbf{x}_{i}^{T}\mathbf{w})^{2}=(\mathbf{y}-\mathbf{X}\mathbf{w})^{T}(\mathbf{y}-\mathbf{X}\mathbf{w}),
$$

and is given by:

$$
\widehat{\mathbf{w}}_{OLS}=(\mathbf{X}^{T}\mathbf{X})^{-1}\;\mathbf{X}^{T}\mathbf{y}
$$

In this notebook, we'll use polynomial regression models to investigate properties and methods that are relevant to more advanced machine learning models. A function implementing the OLS solution is provided as `mod3.ols(x, y, p=1)` from the module `mod3`.

**EXERCISE 1**

The data $\mathcal{D}=\{(x_1,y_1),\ldots,(x_n,y_n)\}$ will be generated with the following procedure:

* $x$ is sampled uniformly in the interval $[a,b]$, i.e. $x\sim U(a,b)$
* $\xi$ is sampled as zero mean gaussian noise, i.e. $\xi\sim\mathcal{N}(0, \sigma^2)$
* the dependent variable $y_i$ is obtained as a sinusoidal function of $x_i$ with additive noise $y_i=f(x_i, \xi_i)=\sin(2\pi x_i) + \xi_i$

Write a function that returns $y$ given values of $x$ and $\xi$.

Write a function that returns $n$ values of $x$ and $y$ given the dataset size $n$ and noise parameters $a$, $b$ and $\sigma$, i.e.
```
x, y = generate_data(n, a, b, sigma)
```
**INSTRUCTIONS**
* use parameters $n=50000$, $a=0$, $b=1$ and $sigma=0.1$
* write the function `f(x, xi=0)`
* write the function `generate_data(n, a, b, sigma)`
* generate the dataset and plot histograms of Numpy arrays `x` and `y`

In [ ]:
def f(x, xi=0):
    
    return y

def generate_data(n, a, b, sigma):
    
    return (x, y)

**EXPECTED OUTPUT**

<img src="fig/poly_fit_histogram.png" style="width:90%;height:90%;display:inline;margin:1px">

**EXERCISE 2**

Generate a dataset $\mathcal{D}=\{\mathbf{x}, \mathbf{y}\}$, and visualise with the data generating function with zero noise term, i.e. $f(x, \xi=0)$.

**INSTRUCTIONS**
* generate a dataset with $n=25$, $a=0$, $b=1$ and $\sigma=0.3$
* plot the dataset and underlying data generating function noise term equal to 0

**EXPECTED OUTPUT**

<img src="fig/poly_fit_sample.png" style="width:50%;height:50%;display:inline;margin:1px">

## Polynomial regression

**EXERCISE 3**

Write a function `g(x, w)` that implements the polynomial model 
$$\mathbf{y}=\mathbf{X}\mathbf{w}$$
for design matrix $\mathbf{X}$ and weights vector $\mathbf{w}$.

Write a function `loss(x, y, w)` that implements the loss function 
$$C(\mathcal{D},\mathbf{w})=\frac{1}{2n}\sum _{i=1}^{n}(\mathbf{x}_{i}^{T}\mathbf{w}-y_{i})^{2}$$
where $\mathbf{x}_{i}^{T}$ is a row vector of $\mathbf{X}$, i.e. a vector containing the $p$ powers of $x_i$.

**INSTRUCTIONS**
* write the function g(x, w)
* write the function loss(x, y, w)
* validate by executing the code below:

```
x = np.array([1.5, 5.3, 3.9, 9.4, 5.5])
y = np.array([4.4, 2.8, 6.1, 7.5, 2.3])
w = np.array([[0.7],
              [0.8],
              [0.9]])

print g(x, w)
print loss(x, y, w)
```

In [ ]:
def g(x, w):
        
    return 

def loss(x, y, w):
    
    return 

**EXPECTED OUTPUT**
```
[  3.925  30.221  17.509  87.744  32.325]
822.2902308
```

**EXERCISE 4**

Fit a polynomial to the generated data, and compare it with the data generating function.

**INSTRUCTIONS**
* generate a dataset with $n=25$, $a=0$, $b=1$ and $\sigma=0.3$
* fit a polynomial of degree $p=4$ to the training data using the function `mod3.ols(x, y, p=deg)`
* plot the generated data, the polynomial and the weights
* you may use the function `mod3.plot_bars` to plot bars representing the weight values

```
mod3.plot_bars(w_ols, title='Weights', ax_labels=('idx', 'w'))```

**EXPECTED OUTPUT**

<img src="fig/poly_fit_weigths.png" style="display:inline;margin:1px">

**EXERCISE 5**

A polynomial of degree $p$ can perfectly fit a dataset with $p+1$ elements.

**INSTRUCTIONS**
* generate a train set of size $n=4$, $a=0$, $b=1$ and $\sigma=0.1$
* train and test polynomials of degree up to $p=6$
* plot the train loss for each polynomial degree

**EXPECTED OUTPUT**

<img src="fig/poly_fit_deg_many.png" style="width:100%;height:100%;display:inline;margin:1px">

**EXERCISE 6**

A polynomial of degree $p$ can perfectly fit a dataset with $p+1$ elements, but in this case it is likely to be fitting the data, rather than the process generating the data, i.e. it is *overfitting*. Model overfitting can be detected by good performance on the train set and much worse performance on the test set.

**INSTRUCTIONS**
* generate a train set of size $n=10$, $a=0$, $b=1$ and $\sigma=0.1$
* generate a test set of size $n=2000$
* train and test polynomials of degree up to $p=10$
* plot the train and test loss for each polynomial degree

**EXPECTED OUTPUT**

<img src="fig/poly_fit_train_test_1.png" style="width: 450px;display:inline;margin:1px"><img src="fig/poly_fit_train_test_2.png" style="width: 450px;display:inline;margin:1px">

**EXERCISE 7**

Cross-validation is a technique to compare between models with the available training data, by partitioning the original dataset into training and test sets, and *rotating* this partitioning. In $k$-fold cross-validation, the original dataset is randomly partitioned into $k$ equal-sized subsets, enabling $k$ evaluations of the models under different partitions of train and test sets. The model with best mean performance and lowest standard deviation is selected.

**INSTRUCTIONS**
* generate a dataset of size $n=2500$, $a=0$, $b=1$ and $\sigma=0.1$
* partition the dataset into $k=10$ partitions
* train and test a polynomial of degree $p=4$ to each partitioning of the dataset. For example, train on partitions 1 to 9, and test on partition 10, then train on partitions 2 to 10, and test on partition 1, etc. 
* plot the loss for each partitioning

**EXPECTED OUTPUT**

<img src="fig/poly_fit_loss_fold.png" style="width:50%;height:50%;display:inline;margin:1px">

**EXERCISE 8**


Evaluate performance of different polynomial degrees under a small training set, by ploting the mean $\mu$ and standard deviation $\sigma$ of the loss (i.e. the rotated test loss).

**INSTRUCTIONS**
* generate a train set of size $n=100$, $a=0$, $b=1$ and $\sigma=0.1$
* fit polynomials of degree up to $p=10$
* evaluate each model with $k=5$ cross-validation
* plot the mean $\mu$ and standard deviation $\sigma$ for each polynomial degree

**EXPECTED OUTPUT**

<img src="fig/poly_fit_deg_1.png" style="width: 450px;display:inline;margin:1px"><img src="fig/poly_fit_deg_2.png" style="width: 450px;display:inline;margin:1px">

**EXERCISE 9**

High degree polynomials have a tendency to wiggle very strongly when fitting small datasets. This is a sign of the model *overfitting* the data. Visualize this effect by varying dataset size $n$, and plotting the polynomial fit and weights.

**INSTRUCTIONS**
* generate diferent train sets of size $n\in\{10, 100, 1000\}$, $a=0$, $b=1$ and $\sigma=0.1$
* fit a polynomial of degree $p=20$ to each train set
* plot the polynomial fit for each dataset size $n$

**EXPECTED OUTPUT**

<img src="fig/poly_fit_reg_data.png" style="width:100%;height:100%;display:inline;margin:1px">

**EXERCISE 10**

The plots of weight values from the previous question do not clearly show that dataset size has a regularizing effect on the norm of the weigths $\mathbf{w}$. Investigate this question by plotting the mean $\mu$ and standard deviation $\sigma$ of the L2 norm of the weights, over $N$ relizations of the sampling and fitting process.

**INSTRUCTIONS**
* generate $N$ train sets of sizes $n\in\{10, 100, 1000, 10000\}$, $a=0$, $b=1$ and $\sigma=0.1$
* fit a polynomial of degree $p=20$ to each train set
* evaluate the mean $\mu$ and standard deviation $\sigma$ of the L2 norm for each train set size $n$
* use function mod3.plot_bars to plot bars representing weight values
* the L2 norm is available in Numpy as `np.linalg.norm`.

**EXPECTED OUTPUT**

<img src="fig/poly_fit_reg_data_stats.png" style="width:50%;height:50%;display:inline;margin:1px">